In [1]:
import torch
import torch.nn as nn
import os
import pandas as pd
import random
import Transformer as tr
import preprocess as pr
import simpleTrajVisualizer as vis

In [2]:
traj_data = pd.read_csv("traj_data_v2.csv")

In [3]:
traj_data.describe()

,Unnamed: 0,Vehicle_ID,X,Y,X_REL,Y_REL
count,307834.00000,307834.000000,307834.000000,307834.000000,3.078340e+05,3.078340e+05
mean,153916.50000,153892.000884,151.187971,-33.894347,4.361413e-07,4.547582e-07
std,88864.16572,88864.166079,0.002386,0.001482,2.208221e-05,1.067770e-05
min,0.00000,0.000000,151.184833,-33.895620,-6.331820e-05,-4.471060e-05
25%,76958.25000,76950.000000,151.186520,-33.895157,-1.109110e-05,-5.980100e-06
50%,153916.50000,153900.000000,151.187506,-33.894778,0.000000e+00,0.000000e+00
75%,230874.75000,230850.000000,151.188627,-33.894293,1.225900e-05,7.062200e-06
max,307833.00000,307800.000000,151.194606,-33.888825,6.605090e-05,4.435870e-05


In [4]:
import random
from sklearn.model_selection import train_test_split

train_data_inputs, test_data_inputs, train_data_targets, test_data_targets, train_indx, test_indx = pr.preprocess_dataset(traj_data, max = 100, max_len = 50, input_len = 20)


In [5]:
if __name__ == "__main__":
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  src_pad_idx = -999
  trg_pad_idx = -999
  src_vocab_size = 100*100 + 100
  trg_vocab_size = 100*100 + 100
  model = tr.Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device, dropout=0.15).to(device)


10100 256
10100 256


In [6]:
criterion = nn.CrossEntropyLoss()
lr = 0.5 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
import math
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = 100*100 + 100
    for batch, i in enumerate(range(0, train_data_inputs.size(0), 1)):
      data = train_data_inputs[i,:,:]
      targets = train_data_targets[i,:,:]
      optimizer.zero_grad()
      output = model(data, targets[:, :])
      loss = criterion(output[0,:,:], targets[0,:])
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
      optimizer.step()
      total_loss += loss.item()
      log_interval = 200
        
    cur_loss = total_loss / train_data_inputs.size(0)
    print('lr {:02.2f} | '
                'loss {:5.2f} | ppl {:8.2f}'.format(
                  scheduler.get_lr()[0],
                  cur_loss, math.exp(cur_loss)))

def evaluate(eval_model, data_source_inputs, data_source_targets):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = 100*100 + 100
    with torch.no_grad():
        for i in range(0, data_source_inputs.size(0), 1):
            data = data_source_inputs[i,:,:]
            targets = data_source_targets[i,:,:]
            output = eval_model(data, targets[:, :])
            output_flat = output
            total_loss += 1 * criterion(output[0,:,:], targets[0,:]).item()
    return total_loss / (data_source_inputs.size(0))

def get_predictions(eval_model, data_source_inputs, data_source_targets):
    eval_model.eval() # Turn on the evaluation mode
    ntokens = 100*100 + 100
    preds_list = []
    target_list = []
    inputs_list = []
    with torch.no_grad():
        for i in range(0, data_source_inputs.size(0), 1):
            data = data_source_inputs[i,:,:]
            targets = data_source_targets[i,:,:]
            output = eval_model(data, targets[:, :])
            preds_list.append(torch.argmax(output[0,:,:], dim=1).tolist())
            target_list.append(targets[0,:].tolist())
            inputs_list.append(data[0,:].tolist())

    return inputs_list, target_list, preds_list

In [7]:
best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None
early_stopping = 0

for epoch in range(1, epochs + 1):
  epoch_start_time = time.time()
  train()
  val_loss = evaluate(model, test_data_inputs, test_data_targets)
  print('-' * 89)
  print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
        'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                    val_loss, math.exp(val_loss)))
  print('-' * 89)

  early_stopping = early_stopping + 1

  if val_loss < best_val_loss:
    early_stopping = 0
    best_val_loss = val_loss
    best_model = model

  if early_stopping > 5:
    break

  scheduler.step()

KeyboardInterrupt: 

In [ ]:
torch.save(best_model, "/content/drive/My Drive/traj_data/trained_model") ## save trained model

In [ ]:
vis.vis_trajectory_scatter(best_model, test_data_inputs, test_data_targets)